To install poincaré from a Jupyter notebook / Google Colab,
run the following command on a cell:
```
!uv pip install poincare
```

# Simulation in poincare
Poincare and SimBio include a number of different options for simulation, including changing solvers and backends, searching for steady states and limit cycles with parameter sweeps and interactive simulation.

### Using different solvers and backends
We can change the method used for simulating the system by passing it to the `solve` method. To use Runge-Kutta  of order 4 (5) instead of the default LSODA (SciPy's implementation of Adams/BDF method):

In [ ]:
import numpy as np

from poincare import (
    Constant,
    Derivative,
    Simulator,
    System,
    Variable,
    assign,
    initial,
)


# Define a System
class DampedOscillator(System):
    x: Variable = initial(default=1)
    v: Derivative = x.derive(initial=0)
    k: Constant = assign(default=1, constant=True)
    eq = v.derive() << -k * x - 0.1 * v

In [ ]:
from poincare import solvers

result = Simulator(DampedOscillator).solve(
    solver=solvers.RK45(),
    save_at=range(3),
)
result

Currently the implemented methods are: Asams/BDF (LSODA), Runge-Kutta of orders 3(2) (RK32), 4(5) (RK45) and 8 (DOP853), Runge-Kutta method of Radau IIA family of order 5 (Radau), and BDF (BDF). All oft them are wrappers to SciPy's implementation  of the method, see [SciPy's documentation](https://docs.scipy.org/doc/scipy/reference/integrate.html) for more details.


Poincare also allows the use of diffent backends to compile the systems to, inculuding [NumPy](https://numpy.org/) (by default) [Numba](https://numba.pydata.org/) and [JAX](https://docs.jax.dev/en/latest/):

In [ ]:
result = Simulator(DampedOscillator, backend="numba").solve(save_at=range(3))
result

### Interactive simulators
We can use the `interact` method create interactive simulations to vary parameters or initial conditions:



In [ ]:
result = Simulator(DampedOscillator).interact(
    save_at=np.linspace(0, 10, 100),
    # values takes a tuple with (start, stop, step)
    values={DampedOscillator.k: (0, 10, 0.1)},
)